In [57]:
import requests
from selenium import webdriver
import pandas as pd
import matplotlib.pyplot as plt
import re
import pickle
import nltk
from nltk.corpus import stopwords
import string
from collections import Counter
from itertools import chain

In [2]:
from ipynb.fs.full.functions import next_page, clean_date, sort_df, find_ngrams, bigramas, freq_bigram  

In [3]:
url = 'https://repositorio.uniandes.edu.co/handle/1992/29/browse?type=organization&value=Departamento+de+Ciencia+Pol%C3%ADtica'

In [21]:
options = webdriver.ChromeOptions()
#Le agregamos argumentos
options.add_argument('--incognito')
#Esto va a abrir un navegador vacío que dice que un software automatizado controla 
driver = webdriver.Chrome(executable_path='../chromedriver', options=options)

driver.get(url)

In [5]:
def obtener_tesis(driver):
    
    #Creamos una lista vacía para que contenga todas las tesis 
    tesis_total = []
    # Obtenemos los impares
    tesis_total.append(driver.find_elements_by_xpath('//li[@class="ds-artifact-item odd"]'))
    # Obtenemos los pares
    tesis_total.append(driver.find_elements_by_xpath('//li[@class="ds-artifact-item even"]'))
    # Transformamos la lista de listas en una sola lista 
    tesis_total =[t for l in tesis_total for t in l]
    return tesis_total

In [6]:
info = []
def obtener_info(driver):
    tesis_totales = obtener_tesis(driver)
    print('iniciando scraping...')
    for tesis in tesis_totales:
        # Obtenemos fecha
        fecha = tesis.find_element_by_xpath('.//span[@class="date"]').text
        # Obtenemos título
        titulo = tesis.find_element_by_xpath('.//h4[@class="artifact-title"]').text
        info_dict={'Fecha': fecha, 'Título': titulo}
        info.append(info_dict)
    return info

In [20]:
def main (driver):
    for x in range (1,22):
        obtener_info(driver)
        next_page(driver)
    return info

In [22]:
main(driver)

iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...


[{'Fecha': '2008',
  'Título': 'La acción colectiva como mecanismo de inclusiónn en los DDHH - el caso de la red juvenil e infantil de promotores de DDHH de Ciudad Bolívar'},
 {'Fecha': '2012',
  'Título': 'Las acciones políticas de mayo del 68 y sus limitaciones - una lectura desde Herbert Marcuse'},
 {'Fecha': '2014',
  'Título': 'La activación de la revocatoria de mandato a nivel municipal en Colombia - lecciones del caso de Bogotá'},
 {'Fecha': '2009',
  'Título': 'Agroindustria y conflicto armado - el caso de la palma de aceite'},
 {'Fecha': '2011',
  'Título': 'Álvaro Uribe: un presidente de teflón - la estrategía de opinión pública que lo hizo inmune a las crísis'},
 {'Fecha': '2008',
  'Título': 'Análisis comparativo del proceso de justicia y paz colombiano - entre las políticas de impunidad y las demandas de justicia'},
 {'Fecha': '2009',
  'Título': 'Análisis de convergencia a través de un indicador sintético de desarrollo para los departamentos de Colombia'},
 {'Fecha': '200

In [23]:
driver.close()

Convertimos el archivo info en un DataFrame

In [24]:
df = pd.DataFrame(info)

Cambiamos los valores dentro de fecha a años

In [25]:
regex = r'(\d{1,4})'
df['Fecha']= df['Fecha'].str.extract(regex).squeeze().str.zfill(4)

In [26]:
df.head()

,Fecha,Título
0,2008,La acción colectiva como mecanismo de inclusió...
1,2012,Las acciones políticas de mayo del 68 y sus li...
2,2014,La activación de la revocatoria de mandato a n...
3,2009,Agroindustria y conflicto armado - el caso de ...
4,2011,Álvaro Uribe: un presidente de teflón - la est...


Volvemos el valor en fecha un integrer

In [27]:
df['Fecha'] = df['Fecha'].astype(int)

In [28]:
df.head()

,Fecha,Título
0,2008,La acción colectiva como mecanismo de inclusió...
1,2012,Las acciones políticas de mayo del 68 y sus li...
2,2014,La activación de la revocatoria de mandato a n...
3,2009,Agroindustria y conflicto armado - el caso de ...
4,2011,Álvaro Uribe: un presidente de teflón - la est...


Limpiamos los títulos de las tesis de signos de puntuación y signos speciales. Además se tokenizan los títulos y se ponen todas las palabras en minúscula

In [53]:
def clean_titulo(df):
    """Esta función recibe un Data Frame y a la columna con """
    # Definimos los signos de puntiación en una variable
    punt_espa = string.punctuation + "–¡¿”“•\r"
    
    # Definimos función para quitar los signos de puntiación 
    def sin_punt(x):
        x = ''.join([i for i in x if i not in (punt_espa)])
        return x
    
    # Aplicamos la función para quitar los signos
    df['Título cleaned'] = df['Título'].apply(sin_punt)
    
    # Definimos las stopwords
    stopwords_esp = stopwords.words('spanish')
    stopwords_esp = stopwords_esp + ['estudio', 'caso', 'análisis', 'bogotá', 'periodo', 'incidencia', 'colombia', 'político', 'hacia', 'impacto', 'the', 'and', 'aproximación', 'influencia', 'factores', 'medio', 'interés', 'programa', 'influencia']
    
    # Ponemos las palabras todas en minúscula y las separamos
    df['Título cleaned'] = df['Título cleaned'].str.lower().str.split()
    
    # Eliminamos las stopwords
    df['Título cleaned'] = df['Título cleaned'].apply(lambda x: [item for item in x if item not in stopwords_esp])
    
    
    return df

In [54]:
clean_titulo(df)

,Fecha,Título,Título cleaned,bigrams
0,2008,La acción colectiva como mecanismo de inclusió...,"[acción, colectiva, mecanismo, inclusiónn, ddh...","[(acción, colectiva), (colectiva, mecanismo), ..."
1,2012,Las acciones políticas de mayo del 68 y sus li...,"[acciones, políticas, mayo, 68, limitaciones, ...","[(acciones, políticas), (políticas, mayo), (ma..."
2,2014,La activación de la revocatoria de mandato a n...,"[activación, revocatoria, mandato, nivel, muni...","[(activación, revocatoria), (revocatoria, mand..."
3,2009,Agroindustria y conflicto armado - el caso de ...,"[agroindustria, conflicto, armado, palma, aceite]","[(agroindustria, conflicto), (conflicto, armad..."
4,2011,Álvaro Uribe: un presidente de teflón - la est...,"[álvaro, uribe, presidente, teflón, estrategía...","[(álvaro, uribe), (uribe, presidente), (presid..."
...,...,...,...,...
415,2006,Transformaciones de las políticas bilaterales ...,"[transformaciones, políticas, bilaterales, seg...","[(transformaciones, políticas), (políticas, bi..."
416,2003,Unión Europea y Estados Unidos: el diseño de s...,"[unión, europea, unidos, diseño, política, ext...","[(unión, europea), (europea, unidos), (unidos,..."
417,2008,Uso del discurso comunicativo del terrorismo e...,"[uso, discurso, comunicativo, terrorismo, peri...","[(uso, discurso), (discurso, comunicativo), (c..."
418,2004,Las UTL en el congreso colombiano: El caso de ...,"[utl, congreso, colombiano, comisiones, primer...","[(utl, congreso), (congreso, colombiano), (col..."


Extraemos los bigramas de los títulos

In [55]:
bigramas(df)

,Fecha,Título,Título cleaned,bigrams
0,2008,La acción colectiva como mecanismo de inclusió...,"[acción, colectiva, mecanismo, inclusiónn, ddh...","[(acción, colectiva), (colectiva, mecanismo), ..."
1,2012,Las acciones políticas de mayo del 68 y sus li...,"[acciones, políticas, mayo, 68, limitaciones, ...","[(acciones, políticas), (políticas, mayo), (ma..."
2,2014,La activación de la revocatoria de mandato a n...,"[activación, revocatoria, mandato, nivel, muni...","[(activación, revocatoria), (revocatoria, mand..."
3,2009,Agroindustria y conflicto armado - el caso de ...,"[agroindustria, conflicto, armado, palma, aceite]","[(agroindustria, conflicto), (conflicto, armad..."
4,2011,Álvaro Uribe: un presidente de teflón - la est...,"[álvaro, uribe, presidente, teflón, estrategía...","[(álvaro, uribe), (uribe, presidente), (presid..."
...,...,...,...,...
415,2006,Transformaciones de las políticas bilaterales ...,"[transformaciones, políticas, bilaterales, seg...","[(transformaciones, políticas), (políticas, bi..."
416,2003,Unión Europea y Estados Unidos: el diseño de s...,"[unión, europea, unidos, diseño, política, ext...","[(unión, europea), (europea, unidos), (unidos,..."
417,2008,Uso del discurso comunicativo del terrorismo e...,"[uso, discurso, comunicativo, terrorismo, peri...","[(uso, discurso), (discurso, comunicativo), (c..."
418,2004,Las UTL en el congreso colombiano: El caso de ...,"[utl, congreso, colombiano, comisiones, primer...","[(utl, congreso), (congreso, colombiano), (col..."


Guardamos el data frame en un archivo pickle para después trabajar con él

In [56]:
df.to_pickle('repo_andes.pkl')